In [12]:
import numpy as np
import pandas as pd

In [ ]:
# folder path containing MSI processed data
folder = "/pathtoMSI/"

# Names of MSI found in folder path
im_names = ["MSIname1","MSIname2"]


In [7]:
# load m/z values of all MSI

peaks = np.genfromtxt(folder + "/file_containg_mz.csv")

# load processed MSI

df = pd.DataFrame()
imstruct = pd.DataFrame()
imstruct["peaks"] = peaks
for i,f in enumerate(im_names):

    msi = np.load(folder+f+"_msi.npy")
    crd = np.load(folder+f+"_crd.npy")
    crd[:,0] =crd[:,0] - np.min(crd[:,0])+1
    crd[:,1] =crd[:,1] - np.min(crd[:,1])+1
    crd = (crd[:,[0,1]]).astype(int)
    
    # load structure score of each ion image
    vec = np.genfromtxt(folder+f+"_spatial_score.csv")
    imstruct[f] = vec
    
    df_msi = pd.DataFrame(np.transpose(msi), columns = peaks )

    df_msi["image"] = np.repeat(f,np.shape(df_msi)[0])
    df_msi["annotation"] = np.repeat(im_names[i],np.shape(df_msi)[0])

    
    df_msi["crd X"] =crd[:,0]
    df_msi["crd Y"] =crd[:,1]

    df = df.append(df_msi, ignore_index = True)

In [9]:
imstruct = imstruct.fillna(0)

# Define the image structure score of the class "Control" and "Interaction" from MSI names
imstruct["Image Structure Control"] = np.max(imstruct[im_names[0]],axis=1)
imstruct["Image Structure Interaction"] = np.max(imstruct[im_names[1]],axis=1)

# Compute image structure fold change
imstruct["Image Fold Change Interaction/Control"]=imstruct["Image Structure Interaction"]/imstruct["Image Structure Control"]

In [10]:
for index, row in imstruct.iterrows():
    if np.isnan(row["Image Structure Control"]) & (row["Image Structure Interaction"] >0):
        imstruct.loc[index]["Image Fold Change Interaction/Control"] = np.inf
        
    if np.isnan(row["Image Structure Interaction"]) & (row["Image Structure Control"] >0):
        imstruct.loc[index]["Image Fold Change Interaction/Control"] = 0

In [ ]:
# save results
imstruct.to_csv("/outputpath/feature_score_msi.csv",index=True)